In [4]:
import os
import json
import pandas as pd
import traceback

In [3]:
!pip install pandas

  Using cached pandas-2.1.4-cp39-cp39-win_amd64.whl.metadata (18 kB)
     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.2 kB 217.9 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/61.2 kB 217.9 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.2 kB 327.7 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 324.6 kB/s eta 0:00:00
  Using cached pytz-2023.3.post1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2023.4-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.1.4-cp39-cp39-win_amd64.whl (10.8 MB)
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.8 MB 1.7 MB/s eta 0:00:10
   ---------------------------------------- 0.2/15.8 MB 2.0 MB/s eta 0:00:08
    --------------------------------------- 0.2/15.8 MB 1.9 MB/s eta 0:00:09
    -----------------------------------

In [6]:
!pip install langchain

  Using cached langchain-0.0.354-py3-none-any.whl.metadata (13 kB)
  Using cached PyYAML-6.0.1-cp39-cp39-win_amd64.whl.metadata (2.1 kB)
  Using cached aiohttp-3.9.1-cp39-cp39-win_amd64.whl.metadata (7.6 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached dataclasses_json-0.6.3-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.0.8-py3-none-any.whl.metadata (7.3 kB)
  Using cached langchain_core-0.1.5-py3-none-any.whl.metadata (4.0 kB)
  Using cached langsmith-0.0.77-py3-none-any.whl.metadata (10 kB)
  Using cached pydantic-2.5.3-py3-none-any.whl.metadata (65 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached multidict-6.0.4-cp39-cp39-win_amd64.whl (28 kB)
  Using cached yarl-1.9.4-cp39-cp39-win_amd64.w

In [13]:
!pip install --upgrade openai

  Using cached openai-1.6.1-py3-none-any.whl.metadata (17 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.26.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached httpcore-1.0.2-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Using cached openai-1.6.1-py3-none-any.whl (225 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.26.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.2-py3-none-any.whl (76 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)


In [17]:
from langchain.chat_models import ChatOpenAI

In [15]:
KEY=os.getenv("OPENAI_API_KEY")

In [19]:
llm =ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)